In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import matplotlib.pyplot as plt
import tikzplotlib
import pickle
import numpy as np

from mujoco_py import MjSim, load_model_from_path

from shared_utils.general import models_folder_path, get_project_root
from soft_tissues_eval.utils import edit_MjModel_cmp_smoothness
from soft_tissues_eval.soft_compression_eval import execute_pre_compression, execute_compression

In [3]:
model_path = os.path.join(models_folder_path(), "soft_tissues", "compression_test_with_solref.xml")
assert os.path.isfile(model_path)


model = load_model_from_path(model_path)

# prefix of the evaluated composite body
cmp_prefix = "cmp_"

# get ref micro element for composite body width
# they are colored in green and blue
GREEN_RGBA = np.array([0.2, 0.8, 0.1, 1], dtype=np.float32)
BLUE_RGBA = np.array([0.1, 0.2, 0.8, 1.], dtype=np.float32)

green_bodies_idx = np.nonzero((model.geom_rgba == GREEN_RGBA).all(axis=1))
blue_bodies_idx = np.nonzero((model.geom_rgba == BLUE_RGBA).all(axis=1))

print(blue_bodies_idx)
print(green_bodies_idx)

assert len(green_bodies_idx[0]) == len(blue_bodies_idx[0]) == 2

# draw uniform samples of solref stiffness
# solref_stiffs = np.arange(-100, -2150, -250)
solref_stiffs = np.array([-200, -975])

solref_damps = np.full_like(solref_stiffs, -111.)
solrefs = np.column_stack((solref_stiffs, solref_damps))

print(solrefs)

(array([155, 156]),)
(array([27, 28]),)
[[-200 -111]
 [-975 -111]]


In [4]:
# cache the compression results to dir
path_to_cache = os.path.join(get_project_root(), "bin", "ouputs", "compression_test")
assert os.path.isdir(path_to_cache)
cache_file_path = os.path.join(path_to_cache, "compression_results.pkl") 

# object containing data
compression_data = {}

if (os.path.isfile(cache_file_path)):
    with open(cache_file_path, "rb") as f:
        compression_data = pickle.load(f)
else:
    for i, solref in enumerate(solrefs):
        compression_data[str(solref)] = {}
        # set solref
        edit_MjModel_cmp_smoothness(model, cmp_prefix, solref)

        # extract useful element indexes
        compressor_body_name = "compressor"
        compressor_body_index = model.body_names.index(compressor_body_name)
        compressor_joint_name = "compressor_slide"
        compressor_joint_index = model.joint_name2id(compressor_joint_name)
        compressor_act_index = 0
        soft_body_name = "test_collider"
        soft_body_index = model.body_names.index(soft_body_name)

        # init simulation
        sim = MjSim(model)
        sim.forward()  # ensure that everything is initialized in the sim object

        print(f"Starting pre compression of soft body {i} ...")

        # execute pre compression
        execute_pre_compression(model, sim, compressor_body_index, compressor_joint_index,
                                compressor_act_index)

        print(f"Starting compression of soft body {i} ...")

        # execute compression
        soft_body_width, force_array = execute_compression(sim, compressor_body_index,
            compressor_act_index, force_step=10., green_bodies_idx=green_bodies_idx, blue_bodies_idx=blue_bodies_idx)

        # save results of the compression
        compression_data[str(solref)]["width"] = soft_body_width
        compression_data[str(solref)]["force"] = force_array
    
    # cache results
    with open(cache_file_path, "wb") as f:
        pickle.dump(compression_data, f, pickle.HIGHEST_PROTOCOL)

In [5]:
generated_pgfs_dir = os.path.join(get_project_root(), "bin", "pgf_plots")
assert os.path.isdir(generated_pgfs_dir)

In [6]:


# plt.figure()
plt.figure(figsize=(10,6))

plt.plot(compression_data[str(solrefs[0])]["width"], compression_data[str(solrefs[0])]["force"])
# plt.xlabel("Soft body width (m)")
# plt.ylabel("Applied force (N)")

plt.show()

# tikzplotlib.save(os.path.join(generated_pgfs_dir, "test.pgf"))

In [7]:
compression_data[str(solrefs[0])]["width"]

array([0.07650206, 0.06954419, 0.06569122, 0.06276741, 0.06027721,
       0.05823661, 0.05649289, 0.05489746, 0.05333882, 0.05176235,
       0.05013591, 0.04850076, 0.04694207, 0.04564313, 0.04460668,
       0.04370606, 0.04286556, 0.0420867 , 0.04134049, 0.04062204,
       0.03993799])

In [8]:
compression_data[str(solrefs[0])]["force"]

array([  1.,  11.,  21.,  31.,  41.,  51.,  61.,  71.,  81.,  91., 101.,
       111., 121., 131., 141., 151., 161., 171., 181., 191., 201.])

In [10]:
plt.plot(np.arange(5), np.arange(5))
plt.show()